# FPL Player Points Prediction Model

This notebook trains a machine learning model to predict Fantasy Premier League player points for upcoming gameweeks.

## Steps:
1. Load and explore the processed FPL data
2. Feature engineering and selection
3. Train multiple regression models
4. Evaluate model performance
5. Save the best model for use in optimization

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Utilities
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Load and Explore Data

In [ ]:
# Load the processed player data
data_path = "../data/processed/fpl_players_latest.csv"

if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f"✅ Data loaded successfully! Shape: {df.shape}")
else:
    print("❌ Data file not found. Please run 'python src/fetch_fpl_data.py' first.")
    raise FileNotFoundError(f"Data file not found at {data_path}")

# Display basic info
print("\nDataset Info:")
print(f"Columns: {len(df.columns)}")
print(f"Rows: {len(df)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Display first few rows
print("First 5 rows:")
display(df.head())

print("\nDataset statistics:")
display(df.describe())

In [ ]:
# Check for missing values
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

if len(missing_data) > 0:
    print("Missing values:")
    print(missing_data)
else:
    print("✅ No missing values found!")

# Position distribution
print("\nPosition distribution:")
print(df['position'].value_counts())

## 2. Exploratory Data Analysis

In [ ]:
# Create visualizations for key variables
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total points distribution
axes[0, 0].hist(df['total_points'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution of Total Points')
axes[0, 0].set_xlabel('Total Points')
axes[0, 0].set_ylabel('Frequency')

# Cost distribution
axes[0, 1].hist(df['cost'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Distribution of Player Cost')
axes[0, 1].set_xlabel('Cost (£m)')
axes[0, 1].set_ylabel('Frequency')

# Form vs Total Points
axes[1, 0].scatter(df['form'], df['total_points'], alpha=0.6, color='coral')
axes[1, 0].set_title('Form vs Total Points')
axes[1, 0].set_xlabel('Form')
axes[1, 0].set_ylabel('Total Points')

# Cost vs Total Points
axes[1, 1].scatter(df['cost'], df['total_points'], alpha=0.6, color='gold')
axes[1, 1].set_title('Cost vs Total Points')
axes[1, 1].set_xlabel('Cost (£m)')
axes[1, 1].set_ylabel('Total Points')

plt.tight_layout()
plt.show()

In [ ]:
# Position-wise analysis
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Box plot of points by position
df.boxplot(column='total_points', by='position', ax=axes[0])
axes[0].set_title('Total Points by Position')
axes[0].set_xlabel('Position')
axes[0].set_ylabel('Total Points')

# Box plot of cost by position
df.boxplot(column='cost', by='position', ax=axes[1])
axes[1].set_title('Cost by Position')
axes[1].set_xlabel('Position')
axes[1].set_ylabel('Cost (£m)')

plt.suptitle('')  # Remove the automatic title
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
# Select numerical columns for correlation
numerical_cols = ['total_points', 'form', 'cost', 'minutes', 'points_per_game', 
                 'influence', 'creativity', 'threat', 'ict_index', 'selected_by_percent',
                 'minutes_per_game', 'cost_efficiency']

# Filter columns that exist in the dataframe
available_cols = [col for col in numerical_cols if col in df.columns]

plt.figure(figsize=(12, 10))
correlation_matrix = df[available_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Heatmap of Key Variables')
plt.tight_layout()
plt.show()

# Show highest correlations with total_points
correlations = correlation_matrix['total_points'].abs().sort_values(ascending=False)
print("\nFeatures most correlated with total_points:")
print(correlations[1:10])  # Exclude self-correlation

## 3. Feature Engineering and Selection

In [ ]:
# Create additional features
df_features = df.copy()

# Additional engineered features
df_features['points_per_minute'] = df_features['total_points'] / (df_features['minutes'] + 1)  # +1 to avoid division by zero
df_features['form_efficiency'] = df_features['form'] / df_features['cost']
df_features['ict_per_cost'] = df_features['ict_index'] / df_features['cost']
df_features['popularity_ratio'] = df_features['selected_by_percent'] / 100

# Goals and assists combined
df_features['goals_assists'] = df_features['goals_scored'] + df_features['assists']

# Defensive actions (for defenders and goalkeepers)
df_features['defensive_actions'] = df_features['clean_sheets'] + df_features['saves']

print("Additional features created:")
print("- points_per_minute")
print("- form_efficiency")
print("- ict_per_cost")
print("- popularity_ratio")
print("- goals_assists")
print("- defensive_actions")

In [ ]:
# Define features for modeling
# Target variable
target = 'form'  # We'll predict form as a proxy for next gameweek points

# Feature selection
feature_columns = [
    'cost', 'total_points', 'points_per_game', 'minutes', 
    'influence', 'creativity', 'threat', 'ict_index',
    'selected_by_percent', 'minutes_per_game', 'cost_efficiency',
    'position_encoded', 'goals_assists', 'defensive_actions',
    'points_per_minute', 'form_efficiency', 'ict_per_cost'
]

# Filter features that exist in the dataframe
available_features = [col for col in feature_columns if col in df_features.columns]

print(f"Selected {len(available_features)} features for modeling:")
for i, feature in enumerate(available_features, 1):
    print(f"{i:2d}. {feature}")

# Prepare data for modeling
X = df_features[available_features].copy()
y = df_features[target].copy()

# Handle any remaining missing values
X = X.fillna(0)
y = y.fillna(0)

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Missing values in X: {X.isnull().sum().sum()}")
print(f"Missing values in y: {y.isnull().sum()}")

## 4. Model Training and Evaluation

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=df_features['position']
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Scale features for some models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define models to test
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, eval_metric='rmse')
}

# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    # Use scaled data for linear models
    if 'Regression' in name:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"  MAE: {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²: {r2:.4f}")
    print()

In [ ]:
# Compare model performance
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'MAE': [results[model]['mae'] for model in results.keys()],
    'RMSE': [results[model]['rmse'] for model in results.keys()],
    'R²': [results[model]['r2'] for model in results.keys()]
})

comparison_df = comparison_df.sort_values('R²', ascending=False)
print("Model Performance Comparison:")
print("=" * 50)
display(comparison_df)

best_model_name = comparison_df.iloc[0]['Model']
print(f"\n🏆 Best performing model: {best_model_name}")

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (name, result) in enumerate(results.items()):
    if i < 5:  # We have 5 models
        y_pred = result['predictions']
        
        # Actual vs Predicted scatter plot
        axes[i].scatter(y_test, y_pred, alpha=0.6)
        axes[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[i].set_xlabel('Actual')
        axes[i].set_ylabel('Predicted')
        axes[i].set_title(f'{name}\nR² = {result["r2"]:.3f}')
        axes[i].grid(True, alpha=0.3)

# Remove the last subplot (we only have 5 models)
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

## 5. Feature Importance Analysis

In [ ]:
# Feature importance for tree-based models
tree_models = ['Random Forest', 'XGBoost']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for i, model_name in enumerate(tree_models):
    if model_name in results:
        model = results[model_name]['model']
        
        if hasattr(model, 'feature_importances_'):
            importance_df = pd.DataFrame({
                'feature': available_features,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=True)
            
            # Plot horizontal bar chart
            axes[i].barh(importance_df['feature'], importance_df['importance'])
            axes[i].set_title(f'{model_name} - Feature Importance')
            axes[i].set_xlabel('Importance')
            
            print(f"\nTop 5 features for {model_name}:")
            top_features = importance_df.tail(5)
            for _, row in top_features.iterrows():
                print(f"  {row['feature']}: {row['importance']:.4f}")

plt.tight_layout()
plt.show()

## 6. Hyperparameter Tuning for Best Model

In [ ]:
# Hyperparameter tuning for the best model
if best_model_name == 'XGBoost':
    print("Tuning XGBoost hyperparameters...")
    
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.2],
        'subsample': [0.8, 1.0]
    }
    
    xgb_model = xgb.XGBRegressor(random_state=42, eval_metric='rmse')
    grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    print(f"Best parameters: {best_params}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
elif best_model_name == 'Random Forest':
    print("Tuning Random Forest hyperparameters...")
    
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    
    rf_model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    print(f"Best parameters: {best_params}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
else:
    # Use the original best model
    best_model = results[best_model_name]['model']
    print(f"Using original {best_model_name} model")

In [ ]:
# Evaluate the final tuned model
if 'Regression' in best_model_name:
    final_predictions = best_model.predict(X_test_scaled)
else:
    final_predictions = best_model.predict(X_test)

final_mae = mean_absolute_error(y_test, final_predictions)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
final_r2 = r2_score(y_test, final_predictions)

print("\nFinal Model Performance:")
print("=" * 30)
print(f"Model: {best_model_name}")
print(f"MAE: {final_mae:.4f}")
print(f"RMSE: {final_rmse:.4f}")
print(f"R²: {final_r2:.4f}")

# Final prediction plot
plt.figure(figsize=(10, 8))
plt.scatter(y_test, final_predictions, alpha=0.6, s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Form')
plt.ylabel('Predicted Form')
plt.title(f'Final Model: {best_model_name}\nR² = {final_r2:.3f}, RMSE = {final_rmse:.3f}')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Save the Model

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the best model
model_path = '../models/model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_model, f)

print(f"✅ Model saved to {model_path}")

# Save model metadata
metadata = {
    'model_name': best_model_name,
    'features': available_features,
    'performance': {
        'mae': final_mae,
        'rmse': final_rmse,
        'r2': final_r2
    },
    'target': target,
    'train_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

metadata_path = '../models/model_metadata.pkl'
with open(metadata_path, 'wb') as f:
    pickle.dump(metadata, f)

print(f"✅ Model metadata saved to {metadata_path}")

# Save feature scaler if needed
if 'Regression' in best_model_name:
    scaler_path = '../models/scaler.pkl'
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"✅ Feature scaler saved to {scaler_path}")

## 8. Model Summary and Next Steps

In [ ]:
print("\n" + "="*60)
print("MODEL TRAINING SUMMARY")
print("="*60)
print(f"✅ Successfully trained and evaluated {len(models)} models")
print(f"🏆 Best model: {best_model_name}")
print(f"📊 Final R² score: {final_r2:.4f}")
print(f"📁 Model saved to: {model_path}")
print(f"🎯 Target variable: {target} (as proxy for expected points)")
print(f"📈 Features used: {len(available_features)}")

print("\nNext Steps:")
print("1. 🔧 Run the optimizer: python src/optimizer.py")
print("2. 🌐 Launch web app: streamlit run web_app/app.py")
print("3. 🔄 Retrain periodically with new gameweek data")
print("="*60)

# Show some example predictions
print("\nExample Predictions (Top 10 by predicted form):")
print("-" * 50)

# Create predictions for all data
if 'Regression' in best_model_name:
    X_scaled = scaler.transform(X)
    all_predictions = best_model.predict(X_scaled)
else:
    all_predictions = best_model.predict(X)

df_with_predictions = df_features.copy()
df_with_predictions['predicted_form'] = all_predictions

top_predictions = df_with_predictions.nlargest(10, 'predicted_form')[['name', 'position', 'team', 'cost', 'form', 'predicted_form']]
display(top_predictions)